# Data Science

We will train 5 different models in 2 experiments, and for each on eadjuste hiperparameters. Also, we will load every run into mlflow.

**Models**
* Logistic Regression
* Random Forest Classifier
* XGBoost Classifier
* Support Vector Classification
* Balanced Random Forest Classifier


### Libraries

In [1]:
from pprint import pprint

import pandas as pd
from imblearn.ensemble import BalancedRandomForestClassifier
from mlflow import MlflowClient
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    average_precision_score,
    classification_report,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier


### Load data

In [ ]:
# Comment once you run this line
%load_ext kedro.ipython
#%reload_kedro

[06/07/25 23:47:10] INFO     Resolved project path as: C:\Users\Esteban\Documents\. Clases_Tec\8.   __init__.py:180
                             Octavo semestre\_Proyecto\kProject\tcaproject.                                        
                             To set a different path, run '%reload_kedro <project_root>'                           

[06/07/25 23:47:11] INFO     Registering new custom resolver: 'km.random_name'                    mlflow_hook.py:65

                    INFO     Kedro is sending anonymous usage data with the sole purpose of improving plugin.py:233
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[06/07/25 23:47:12] INFO     Kedro project tcaProject                                               __init__.py:146

                    INFO     Defined global variable 'context', 'session', 'catalog' and            __init__.py:147
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        __init__.py:153

In [13]:
df_exp1 = catalog.load("reservaciones_exp1") # type: ignore # noqa: F821
df_exp2 = catalog.load("reservaciones_exp2") # type: ignore # noqa: F821


[06/07/25 23:47:18] INFO     Loading data from reservaciones_exp1 (ParquetDataset)...           data_catalog.py:403

                    INFO     Loading data from reservaciones_exp2 (ParquetDataset)...           data_catalog.py:403

### EDA

In [14]:
df_exp1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 202491 entries, 0 to 203001
Data columns (total 17 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   numero_personas                202491 non-null  float64
 1   numero_adultos                 202491 non-null  float64
 2   numero_menores                 202491 non-null  float64
 3   numero_noches                  202491 non-null  float64
 4   total_habitaciones             202491 non-null  float64
 5   total_habitaciones_empresa     202491 non-null  float64
 6   cupo_tipo_habitacion           202491 non-null  float64
 7   reservacion_pendiente          202491 non-null  float64
 8   total_tarifa                   202491 non-null  float64
 9   reservacion                    202491 non-null  float64
 10  cancelacion                    202491 non-null  float64
 11  dias_llegada-reservacion       202491 non-null  float64
 12  dias_salida-llegada            2024

In [15]:
df_exp2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 199813 entries, 0 to 203001
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   nombre_paquete          199813 non-null  float64
 1   ciudad_agencia          199813 non-null  float64
 2   nombre_tipo_habitacion  199813 non-null  float64
 3   nombre_canal            199813 non-null  float64
 4   nombre_estado           199813 non-null  float64
 5   cancelacion             199813 non-null  int64  
dtypes: float64(5), int64(1)
memory usage: 10.7 MB


In [16]:
print(df_exp1.isna().sum().sum())
print(df_exp2.isna().sum().sum())

0
0


In [11]:
df_exp2.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Index: 199813 entries, 0 to 203001
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   nombre_paquete          199813 non-null  float64
 1   ciudad_agencia          199813 non-null  float64
 2   nombre_tipo_habitacion  199813 non-null  float64
 3   nombre_canal            199813 non-null  float64
 4   nombre_estado           199813 non-null  float64
 5   cancelacion             199813 non-null  int64  
dtypes: float64(5), int64(1)
memory usage: 10.7 MB
